In [1]:
import gymnasium as gym
import numpy as np
import torch
torch.set_printoptions(threshold=2000, linewidth=200)
np.set_printoptions(threshold=2000, linewidth=200)

environmentName = "CarRacing-v3"
renderMode = "rgb_array"
numEpisodes = 1
stepCountLimit = 256

env = gym.make(environmentName, render_mode=renderMode)
observationShape = torch.tensor(env.observation_space.shape)
actionSize = torch.tensor(env.action_space.shape) if hasattr(env.action_space, 'shape') else np.array([env.action_space.n])
print(f"Env {environmentName} with observations {observationShape} and actions {actionSize}")

for episode in range(numEpisodes):
    observation, info = env.reset()
    observations, actions, rewards, dones = [observation], [], [], []
    stepCount, totalReward, done = 1, 0, False
    while not done:
        action = env.action_space.sample() # For now random action
        observation, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated or stepCount > stepCountLimit
        totalReward += reward
        
        observations.append(observation)
        actions.append(action)
        rewards.append(reward)
        dones.append(done)

        stepCount += 1
        env.render()

    print(f"Total Reward: {totalReward:.4f}")
env.close()

Env CarRacing-v3 with observations tensor([96, 96,  3]) and actions tensor([3])
Total Reward: -4.0282


In [7]:
from PIL import Image
image = Image.fromarray(observation[:-12,:,:])
image.save('test2.png')

In [3]:
data = torch.randn(96, 96, 3)  # Example tensor
inverted_data = data.permute(2, 0, 1)  # Move channels to the front

print(inverted_data.shape)

torch.Size([3, 96, 96])
